# Imágenes referencia de cebada - cargado y conversión jpg

por ahora solo una réplica del control 2 - 

acordarse de balance de histograma (en canal blanco-negro) cuando se vayan a predecir conteos en otras imágenes con otros parámetros (brillo - contraste)

In [4]:
# importar librerías
import cv2
import numpy as np
from PIL import Image, ImageTk
import os
Image.MAX_IMAGE_PIXELS = None

In [3]:
dirname = "Z:\\CRISTINA\\16.10.23 Cultivo Hv + CIB123 1 uM\\Control\\2"

In [4]:
out_dirname = "C:\\Users\\Usuario\\Desktop\\cebada test"

In [7]:
for file_path in os.listdir(dirname): #
    # Leer la imagen
    if "Metadata" not in file_path:
        image = Image.open(dirname+"\\"+str(file_path))
        Z = np.array(image, dtype="uint8")
        cv2.imwrite(out_dirname+"\\"+str(file_path)+".png",Z)

Balance histograma

In [6]:
from skimage import data
from skimage import exposure
from skimage.exposure import match_histograms

In [ ]:
dirname_i = ""
outdirname_i = ""

ref_i = ""

ref_image = Image.open(ref_i)
ref_image = np.array(ref_image, dtype="uint8")

for file_path in os.listdir(dirname_i): #
    if "Metadata" not in file_path:
        image_tomatch = Image.open(dirname_i+"\\"+str(file_path))
        image_tomatch = np.array(image_tomatch, dtype="uint8")
        matched = match_histograms(image_tomatch, ref_image, channel_axis=None) #grayscale image
        cv2.imwrite(outdirname_i+"\\"+str(file_path)+".png",matched)

# PROCESAMIENTO DE MASCARAS RESULTANTES JSON

Import

In [5]:
import json

# Reading JSON file

In [10]:
dirmasks = "C:\\Users\\Usuario\\Desktop\\cebada test\\train\\mascaras_json\\3samplesbarley.json"

In [16]:
f = open(dirmasks, "r")
data = json.load(f)
data = data["_via_img_metadata"]

In [27]:
img_dir = "C:\\Users\\Usuario\\Desktop\\cebada test\\train\\images"
mask_dir = "C:\\Users\\Usuario\\Desktop\\cebada test\\train\\mask"

In [33]:
for key, value in data.items():
    
    filename = value["filename"]
    print(filename)
    img_path = f"{img_dir}\\{filename}"
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    
    h, w =img.shape
    
    
    regions = value["regions"]

    microspore_counter = 0
    proembryo_counter = 0

    # per region, segment and create a mask
    for region in regions:
        
        # if proembryo
        if region["region_attributes"]["name"] == "proembryo":
            
            mask = np.zeros((h,w))
            
            proembryo_counter = proembryo_counter + 1
            
            shape_attributes = region["shape_attributes"]
            xc = shape_attributes["cx"]
            yc = shape_attributes["cy"]
            radius = int(shape_attributes["r"])
            
            cv2.circle(mask, (xc,yc) , radius, 255, -1)
            cv2.imwrite(f"{mask_dir}\\{filename}_PROEMBRYO_{proembryo_counter}.png", mask)

            mask = np.zeros((h,w))
        
        # if microspore
        if region["region_attributes"]["name"] == "microspore":

            mask = np.zeros((h,w))
            microspore_counter = microspore_counter + 1
            
            shape_attributes = region["shape_attributes"]
            xpoints = shape_attributes["all_points_x"]
            ypoints = shape_attributes["all_points_y"]
    
            contours = []
            for x , y in zip(xpoints, ypoints):
                contours.append((x,y))
            contours = np.array(contours)
        
            cv2.drawContours(mask, [contours], -1, 255, -1)
            cv2.imwrite(f"{mask_dir}\\{filename}_MICROSPORE_{microspore_counter}.png", mask)

            mask = np.zeros((h,w))
        
        

c.tif.png
c0001.tif.png
c0003.tif.png


UNET MASK FORMAT
0 FOR BACKGROUND
1 FOR MICROSPORE
2 FOR PROEMBRYO

IN EACH PIXEL

In [55]:
for key, value in data.items():
    
    filename = value["filename"]
    print(filename)
    img_path = f"{img_dir}\\{filename}"
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    
    h, w =img.shape
    
    mask = np.zeros((h,w))
    
    regions = value["regions"]

    microspore_counter = 0
    proembryo_counter = 0

    # per region, segment and create a mask
    for region in regions:
        
        # if proembryo
        if region["region_attributes"]["name"] == "proembryo":
            
            
            
            proembryo_counter = proembryo_counter + 1
            
            shape_attributes = region["shape_attributes"]
            xc = shape_attributes["cx"]
            yc = shape_attributes["cy"]
            radius = int(shape_attributes["r"])
            
            cv2.circle(mask, (xc,yc) , radius, 200, -1)
            

            
        
        # if microspore
        if region["region_attributes"]["name"] == "microspore":

            
            microspore_counter = microspore_counter + 1
            
            shape_attributes = region["shape_attributes"]
            xpoints = shape_attributes["all_points_x"]
            ypoints = shape_attributes["all_points_y"]
    
            contours = []
            for x , y in zip(xpoints, ypoints):
                contours.append((x,y))
            contours = np.array(contours)
        
            cv2.drawContours(mask, [contours], -1, 255, -1)


    # pass all microspore values to 1, background is left as 0
    ind_x, ind_y = np.where(mask ==255, )
    mask[ind_x, ind_y]=1

    
    # pass all proembryo values to 2
    ind_x, ind_y = np.where(mask ==200, )
    mask[ind_x, ind_y]=2
    
    
    cv2.imwrite(f"{mask_dir}\\{filename}", mask)

    mask = np.zeros((h,w))

            
        
        

c.tif.png
c0001.tif.png
c0003.tif.png


In [10]:
dirmasks = "C:\\Users\\Usuario\\Desktop\\cebada test\\train\\mascaras_json\\test2.json"

In [11]:
f = open(dirmasks, "r")
data = json.load(f)
data = data["_via_img_metadata"]

In [12]:
img_dir = "C:\\Users\\Usuario\\Desktop\\cebada test\\test\\images"
mask_dir = "C:\\Users\\Usuario\\Desktop\\cebada test\\test\\mask"

In [13]:
for key, value in data.items():
    
    filename = value["filename"]
    print(filename)
    img_path = f"{img_dir}\\{filename}"
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    
    h, w =img.shape
    
    mask = np.zeros((h,w))
    
    regions = value["regions"]

    microspore_counter = 0
    proembryo_counter = 0

    # per region, segment and create a mask
    for region in regions:
        
        # if proembryo
        if region["region_attributes"]["name"] == "proembryo":
            
            
            
            proembryo_counter = proembryo_counter + 1
            
            shape_attributes = region["shape_attributes"]
            xc = shape_attributes["cx"]
            yc = shape_attributes["cy"]
            radius = int(shape_attributes["r"])
            
            cv2.circle(mask, (xc,yc) , radius, 200, -1)
            

            
        
        # if microspore
        if region["region_attributes"]["name"] == "microspore":

            
            microspore_counter = microspore_counter + 1
            
            shape_attributes = region["shape_attributes"]
            xpoints = shape_attributes["all_points_x"]
            ypoints = shape_attributes["all_points_y"]
    
            contours = []
            for x , y in zip(xpoints, ypoints):
                contours.append((x,y))
            contours = np.array(contours)
        
            cv2.drawContours(mask, [contours], -1, 255, -1)


    # pass all microspore values to 1, background is left as 0
    ind_x, ind_y = np.where(mask ==255, )
    mask[ind_x, ind_y]=1

    
    # pass all proembryo values to 2
    ind_x, ind_y = np.where(mask ==200, )
    mask[ind_x, ind_y]=2
    
    
    cv2.imwrite(f"{mask_dir}\\{filename}", mask)

    mask = np.zeros((h,w))

            
        
        

c0002.tif.png
c0004.tif.png
c0005.tif.png
c0010.tif.png
c0011.tif.png
c0012.tif.png
c0017.tif.png
c0018.tif.png
c0019.tif.png
c0006.tif.png
c0007.tif.png
c0008.tif.png
c0009.tif.png
c0013.tif.png
c0014.tif.png
c0015.tif.png
c0016.tif.png
